In [1]:
# libraries
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta

In [ ]:
# data size: 52 data files, each is about 25MB. Total data size is 1.25GB.
# processing data: load one data file at a time, group data by subway stations and days of the week to calculate ridership, then
# calculate 52 week mean.

In [33]:
# experiment with one data file first 
df = pd.read_csv("data/turnstile_171223.txt")
print df.shape
df.head()

(199474, 11)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/16/2017,03:00:00,REGULAR,6446400,2181796
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/16/2017,07:00:00,REGULAR,6446408,2181807
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/16/2017,11:00:00,REGULAR,6446457,2181889
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/16/2017,15:00:00,REGULAR,6446670,2181964
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/16/2017,19:00:00,REGULAR,6446975,2182026


In [37]:
# use every midnight (00:00:00 - 00:03:59)
def midnight(s):    
    if s[0:2]=='01' or s[0:2]=='02' or s[0:2]=='03' or s[0:2]=='00':
        return 'YES'
    else:
        return s    

df['TIME'] = df['TIME'].apply(midnight)
df_midnight = df[(df['TIME'] == 'YES')]

In [12]:
print df.shape
print df_midnight.shape

(199474, 11)
(32582, 11)


In [4]:
df_midnight['DATE'] = pd.to_datetime(df_midnight['DATE'],format='%m/%d/%Y')
df_midnight['WEEKDAY'] = df_midnight['DATE'].apply(lambda x: x.weekday())

C:\Users\jhuan\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\jhuan\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [30]:
df_midnight.sort_values(by = ['STATION', 'DATE'])[:5]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEKDAY
29856,H007,R248,00-00-00,1 AV,L,BMT,2017-12-16,YES,REGULAR,13021054,14515138,5
29898,H007,R248,00-00-01,1 AV,L,BMT,2017-12-16,YES,REGULAR,58137408,36931419,5
29940,H007,R248,00-03-00,1 AV,L,BMT,2017-12-16,YES,REGULAR,370185490,387175207,5
29982,H007,R248,00-03-01,1 AV,L,BMT,2017-12-16,YES,REGULAR,2079648,872209,5
30024,H007,R248,00-03-02,1 AV,L,BMT,2017-12-16,YES,REGULAR,5803812,487215,5


In [31]:
df_midnight.sort_values(by = ['STATION', 'DATE'])[-5:]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEKDAY
175586,R419,R326,00-00-01,ZEREGA AV,6,IRT,2017-12-22,YES,REGULAR,282345,448440,4
175628,R419,R326,00-03-00,ZEREGA AV,6,IRT,2017-12-22,YES,REGULAR,584785,271478,4
175670,R419,R326,00-03-01,ZEREGA AV,6,IRT,2017-12-22,YES,REGULAR,285105,52921,4
175712,R419,R326,00-05-00,ZEREGA AV,6,IRT,2017-12-22,YES,REGULAR,106,0,4
175754,R419,R326,00-05-01,ZEREGA AV,6,IRT,2017-12-22,YES,REGULAR,39,84,4


In [23]:
df_entries_grouped = df_midnight.groupby(['STATION', 'DATE']).agg({'ENTRIES':np.sum})

In [24]:
df_entries_grouped[:10]

ENTRIES
STATION DATE                  
1 AV    2017-12-16  3042691118
        2017-12-17  3042710114
        2017-12-18  3042723264
        2017-12-19  3042744916
        2017-12-20  3042767332
        2017-12-21  3042789542
        2017-12-22  3042811366
103 ST  2017-12-16   147423304
        2017-12-17   147437743
        2017-12-18   147448711

In [18]:
# Appendix 1: How many unique stations names are in this dataset?
print len(df['STATION'].unique())
print df['STATION'].unique()

373
['59 ST' '5 AV/59 ST' '57 ST-7 AV' '49 ST' 'TIMES SQ-42 ST'
 '34 ST-HERALD SQ' '28 ST' '23 ST' '14 ST-UNION SQ' '8 ST-NYU' 'PRINCE ST'
 'CANAL ST' 'CITY HALL' 'CORTLANDT ST' 'RECTOR ST' 'WHITEHALL S-FRY'
 'DELANCEY/ESSEX' 'BOWERY' 'CHAMBERS ST' 'FULTON ST' 'BROAD ST' '7 AV'
 'PARK PLACE' 'BOTANIC GARDEN' 'PROSPECT PARK' 'PARKSIDE AV' 'CHURCH AV'
 'BEVERLEY ROAD' 'CORTELYOU RD' 'NEWKIRK PLAZA' 'AVENUE H' 'AVENUE J'
 'AVENUE M' 'KINGS HWY' 'AVENUE U' 'NECK RD' 'SHEEPSHEAD BAY'
 'BRIGHTON BEACH' 'OCEAN PKWY' 'BOROUGH HALL' 'JAY ST-METROTEC' 'DEKALB AV'
 'ATL AV-BARCLAY' 'UNION ST' '4AV-9 ST' 'PROSPECT AV' '25 ST' '36 ST'
 '45 ST' '53 ST' 'BAY RIDGE AV' '77 ST' '86 ST' 'BAY RIDGE-95 ST' '8 AV'
 'FT HAMILTON PKY' 'NEW UTRECHT AV' '18 AV' '20 AV' 'BAY PKWY' '9 AV'
 '50 ST' '55 ST' '71 ST' '79 ST' '25 AV' 'BAY 50 ST' 'CONEY IS-STILLW'
 'W 8 ST-AQUARIUM' '6 AV' '3 AV' '1 AV' 'BEDFORD AV' 'LORIMER ST'
 'GRAHAM AV' 'GRAND ST' 'MONTROSE AV' 'MORGAN AV' 'JEFFERSON ST'
 'MYRTLE-WYCKOFF' 'HALSEY

In [ ]:
# Appendix 2:
# MTA turnstile data filename format: "turnstile_" + Saturday date, for instance, first file's name is "turnstile_170107"
# Define Function to generate Saturday dates in 2017 so we have a list of turnstile data filenames
# Modified code on Stack Overflow: https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python

def saturdays2017():
    d = date(2017, 1, 7)                    # First Saturday in 2017 - January 7th
    while d.year == 2017:
        yield d
        d += timedelta(days = 7)        
filenames = list()
for d in saturdays2017():
    filenames.append("turnstile_%s" % d.strftime("%y%m%d"))
print filenames